In [18]:
import pandas as pd
import time as time
import random
import os
from urllib.parse import urlparse, urljoin
from datetime import datetime
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from azure.storage.blob import BlobServiceClient
from config import new_chromedriver_path, user_agents, path, connection_string
from web_scraper import WebScraper
from vpn import VPNChanger

In [ ]:
lista_sites = pd.read_csv('C:/Users/victo/AI/Cognificadora/cognificar_leiloes/lista_sites.csv', sep=';')
lista_sites = lista_sites['0'].tolist()

In [ ]:
df_sodero = pd.read_csv('df_sodero.csv')
df_lista = df_sodero['Leilao'].apply(lambda x: urljoin(lista_sites[0], x))

In [ ]:
scraper = WebScraper(new_chromedriver_path, user_agents, path)
# vpn_changer = VPNChanger()
# vpn_files = vpn_changer.get_vpn_files()

In [ ]:
if os.path.exists('bens.csv'):
    os.remove('bens.csv')
if os.path.exists('infos_especificas.csv'):
    os.remove('infos_especificas.csv')
if os.path.exists('df_intermediario.csv'):
    os.remove('df_intermediario.csv')
if os.path.exists('df_leilao_aberto.csv'):
    os.remove('df_leilao_aberto.csv')
if os.path.exists('df_leilao_encerrado.csv'):
    os.remove('df_leilao_encerrado.csv')
if os.path.exists('df_concat.csv'):
    os.remove('df_concat.csv')
if os.path.exists('control_home.txt'):
    os.remove('control_home.txt')
if  os.path.exists('control_pag.txt'):
    os.remove('control_pag.txt')

In [ ]:
def scrape_aberto(lista_bens):
    df_intermediario = pd.DataFrame()
    infos_base = []
    for inf in lista_bens:
        titulo = inf.find('h2', class_='titulo')
        titulo = titulo.text if titulo else None

        link = inf.find('a', class_='link')
        link = link.get('href') if link else None

        imagem = inf.find('img', class_='imagemDisponivel img-responsive')
        imagem = imagem.get('src') if imagem else None

        status_lote = inf.find('div', class_='statusLote statusLote-Aguardando')
        status_lote = status_lote.text if status_lote else None

        lance_atual = inf.find('span', class_='lance-atual')
        lance_atual = lance_atual.get('title') if lance_atual else None

        visitas_lances = inf.find('span', class_='visitas-lances')
        visitas_lances = visitas_lances.text if visitas_lances else None

        infos_base.append(
            {
                'Titulo': titulo,
                'Imagem': imagem,
                'Link': link,
                'Status Lote': status_lote,
                'Lance Atual': lance_atual,
                
                'Visitas_lances': visitas_lances
            }
        )
        
        df_bens = pd.DataFrame(infos_base).replace(r'\n|\t', '', regex=True)
        df_bens.to_csv('bens.csv', index=False)
    if os.path.exists('df_intermediario.csv'):
        df_intermediario = pd.read_csv('df_intermediario.csv')
    df_intermediario = pd.concat([df_intermediario, df_bens], axis=0)
    df_intermediario.to_csv('df_intermediario.csv', index=False)

    return df_intermediario

In [ ]:
import subprocess
import psutil

def find_chrome_processes():
    chrome_processes = []
    for proc in psutil.process_iter(['pid', 'name']):
        if proc.info['name'] == 'chrome.exe':
            chrome_processes.append(proc)
    return chrome_processes

def kill_chrome_processes():
    chrome_processes = find_chrome_processes()
    for process in chrome_processes:
        try:
            subprocess.run(['taskkill', '/F', '/PID', str(process.pid)], check=True)
        except subprocess.CalledProcessError as e:
            pass

kill_chrome_processes()

In [ ]:
start_index = scraper.get_current_index("control_home")
# vpn_changer.connect_vpn(vpn_files[random.randint(0, len(vpn_files) - 1)])

df_leilao_aberto = pd.DataFrame()
try:
    for x in range(start_index, len(df_lista)):
        scraper.driver = scraper.header_configuration(headless=True)   
        time.sleep(random.randint(13,16))
        f = df_lista.iloc[x]
        
        # if x % 5 == 0 and x > 0:
        #    vpn_changer.change_ip()

        scraper.driver.get(f'{f}ordenacao/nu_lote/tipo-ordenacao/crescente/qtde-itens/{random.randint(4000,6000)}/')
        scraper.driver.delete_all_cookies()

        print(scraper.driver.current_url)
        if "validate.perfdrive.com" in urlparse(scraper.driver.current_url).netloc:
            scraper.driver = scraper.scrape_with_captcha_retry(scraper.driver.current_url, f'{f}ordenacao/nu_lote/tipo-ordenacao/crescente/qtde-itens/{random.randint(4000,6000)}/') #, df_lista
        if 'encerrado' in scraper.driver.current_url:
            continue

        soup = BeautifulSoup(scraper.driver.page_source, 'html.parser')
        
        try:
            span = soup.find('span', class_='textoAux2')
            if span.text == 'Leilão não encontrado!':
                continue
        except:
            pass

        ul_paginas = soup.find('ul', class_='tipo-vizualizacao visual_imagemlista')
        lista_bens = ul_paginas.find_all('li', attrs={'guardar': True})

        df_aberto = scrape_aberto(lista_bens)
        if os.path.exists('df_leilao_aberto.csv'):
            df_leilao_aberto = pd.read_csv('df_leilao_aberto.csv')
        df_leilao_aberto = pd.concat([df_leilao_aberto, df_aberto], axis=0)
        df_leilao_aberto.drop_duplicates().to_csv('df_leilao_aberto.csv', index=False)

        print(f"Index {x + 1} of {len(df_lista)}")
        scraper.update_current_index(x + 1, "control_home")
        scraper.driver.quit()
        
        # scraper.kill_chrome_processes()
        scraper.delete_cache(get_previous_url=False)
        
    pd_ajuste = pd.read_csv('df_leilao_aberto.csv')
    pd_ajuste['link_clicavel'] = pd_ajuste['Link'].apply(lambda x: urljoin(lista_sites[0], x))
    pd_ajuste['data_hora_atualizacao'] = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    pd_ajuste.to_csv('df_leilao_aberto.csv', index=False)
    # vpn_changer.disconnect_vpn()
    print(f'Finalizado as {datetime.now().strftime("%d/%m/%Y %H:%M:%S")}')
finally:
    try:
        # vpn_changer.disconnect_vpn()
        scraper.driver.service.process.terminate()
        scraper.driver.quit()
        scraper.delete_cache(get_previous_url=False)
    except:
        pass

In [ ]:
# enviar para o blob do azure
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
blob_client = blob_service_client.get_blob_client("leilao-sodre", "df_leilao_aberto")

with open('df_leilao_aberto.csv', "rb") as data:
    blob_client.upload_blob(data, overwrite=True)

In [27]:
# import time

scraper.driver = scraper.header_configuration(headless=False)
scraper.driver.get('https://www.superbid.net/todos-eventos?filter=modalityId:[1,4];subMarketplaces.id:all&byPage=marketPlacePage&pageNumber=1&pageSize=100000')

soup = BeautifulSoup(scraper.driver.page_source, 'html.parser')
time.sleep(30)
# WebDriverWait(scraper.driver, 30).until(EC._element_if_visible((By.XPATH, "//div[@class='MuiGrid-root MuiGrid-container jss93 css-1hk9k3n']")))
div_geral = soup.find_all('div', class_='MuiGrid-root MuiGrid-item MuiGrid-grid-xs-auto css-1gd6ahq')

informacao_superbid = []
for div in div_geral:
    # get the information from personalized attribute data-auction-name
    auction_name = div['data-auction-name']

    informacao_superbid.append(
        {
            'leilao': auction_name
        }
    )

df_superbid = pd.DataFrame(informacao_superbid)
df_superbid


""


In [28]:
div_geral


[]